In [237]:
#generate summary info for CuBIDs validator

#import packages
import pandas as pd
import numpy as np
import os

In [238]:
#set path:
path='/cbica/projects/wolf_satterthwaite_reward/Margaret/Day2/curation/code/sandbox/validator_outputs'
bids_path='/cbica/projects/wolf_satterthwaite_reward/Margaret/Day2/curation/BIDS'
iteration='1.2'
prefix='d2_r2'

#define df
df=pd.read_csv(path+'/iteration'+iteration+'/'+prefix+'_validation.csv')

In [239]:
num_er=np.sum(df['severity']=='error')
print(num_er)

0


In [240]:
num_war=np.sum(df['severity']=='warning')
print(num_war)

497


In [241]:
df_war=df[df['severity']=='warning']

In [242]:
war_type=df_war['type'].unique()
print(war_type)

['EVENTS_TSV_MISSING' 'README_FILE_MISSING' 'NO_AUTHORS']


In [243]:
for warning in war_type:
    num_war_type=np.sum(df_war['type']==warning)
    descrip=df_war.loc[df['type'] == warning, 'description'].unique()
    dprint=' '.join(map(str, descrip))
    print(warning,'(',dprint,')', ':', num_war_type, 'subjects')

EVENTS_TSV_MISSING ( Task scans should have a corresponding events.tsv file. If this is a resting state scan you can ignore this warning or rename the task to include the word "rest". ) : 495 subjects
README_FILE_MISSING ( The recommended file /README is missing. See Section 03 (Modality agnostic files) of the BIDS specification. ) : 1 subjects
NO_AUTHORS ( The Authors field of dataset_description.json should contain an array of fields - with one author per field. This was triggered because there are no authors, which will make DOI registration from dataset metadata impossible. ) : 1 subjects


##searching subject data for participants with more than one T1w

In [244]:
df_files=pd.read_csv(path+'/iteration'+iteration+'/'+prefix+'_files.csv')

In [245]:
df_anat=df_files[df_files['Modality']=='anat']

In [246]:
df_anat.count()

KeyParamGroup                     128
KeyGroup                          128
ParamGroup                        128
Counts                            128
Dim1Size                          128
Dim2Size                          128
Dim3Size                          128
EchoTime                          128
EffectiveEchoSpacing                0
FlipAngle                         128
HasFieldmap                       128
KeyGroupCount                     128
Modality                          128
NSliceTimes                       128
NumVolumes                        128
Obliquity                         128
ParallelReductionFactorInPlane     69
PartialFourier                    128
PhaseEncodingDirection              0
RepetitionTime                    128
TotalReadoutTime                    0
UsedAsFieldmap                    128
VoxelSizeDim1                     128
VoxelSizeDim2                     128
VoxelSizeDim3                     128
FilePath                          128
dtype: int64

In [247]:
df_anat=df_anat[['KeyParamGroup','ParamGroup','Counts','FilePath']]

In [248]:
df_anat['Subject']=df_anat['FilePath'].str.split('\/').str[10]

In [249]:
df_anat['Multiple_T1']=df_anat.duplicated(subset=['Subject'])

In [250]:
df_repeat=df_anat[df_anat['Multiple_T1']==True]
df_repeat.count()

KeyParamGroup    3
ParamGroup       3
Counts           3
FilePath         3
Subject          3
Multiple_T1      3
dtype: int64

In [251]:
#df_repeat

In [252]:
#finding acquisition group(s) for subj. with T2 scans
df_acq=pd.read_csv(path+'/iteration'+iteration+'/'+prefix+'_AcqGrouping.csv')
df_repeat_acq=pd.merge(df_repeat, df_acq, left_on = 'Subject', right_on = 'subject', how = 'inner')
df_repeat_acq=df_repeat_acq[['AcqGroup']]
df_repeat_acq

,AcqGroup
0,3
1,3
2,3


In [253]:
#searching subject data for participants with more than one fmap

df_fmap=df_files[df_files['Modality']=='fmap']

In [254]:
df_fmap=df_fmap[['KeyParamGroup','KeyGroup','ParamGroup','Counts','FilePath']]
df_fmap['Subject']=df_fmap['FilePath'].str.split('\/').str[10]

In [255]:
df_keygr=df_fmap['KeyGroup'].unique()
print(df_keygr)

['datatype-fmap_fmap-phase1_suffix-phase1'
 'datatype-fmap_fmap-phase2_suffix-phase2'
 'datatype-fmap_fmap-magnitude1_suffix-magnitude1'
 'datatype-fmap_fmap-magnitude2_suffix-magnitude2']


In [256]:
df_p1=df_fmap[df_files['KeyGroup']=='datatype-fmap_fmap-phase1_suffix-phase1']
df_p1['Multiple_P1']=df_p1.duplicated(subset=['Subject'])

/scratch/wolfsatterthwaitereward/tmp.WL1HnBKnz7/ipykernel_38573/1386749628.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_p1=df_fmap[df_files['KeyGroup']=='datatype-fmap_fmap-phase1_suffix-phase1']
/scratch/wolfsatterthwaitereward/tmp.WL1HnBKnz7/ipykernel_38573/1386749628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_p1['Multiple_P1']=df_p1.duplicated(subset=['Subject'])


In [257]:
df_p1r=df_p1[df_p1['Multiple_P1']==True]
df_p1r.count()

KeyParamGroup    0
KeyGroup         0
ParamGroup       0
Counts           0
FilePath         0
Subject          0
Multiple_P1      0
dtype: int64

In [258]:
df_p2=df_fmap[df_files['KeyGroup']=='datatype-fmap_fmap-phase2_suffix-phase2']
df_p2['Multiple_P2']=df_p2.duplicated(subset=['Subject'])
df_p2r=df_p2[df_p2['Multiple_P2']==True]
df_p2r.count()

/scratch/wolfsatterthwaitereward/tmp.WL1HnBKnz7/ipykernel_38573/3665166110.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_p2=df_fmap[df_files['KeyGroup']=='datatype-fmap_fmap-phase2_suffix-phase2']
/scratch/wolfsatterthwaitereward/tmp.WL1HnBKnz7/ipykernel_38573/3665166110.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_p2['Multiple_P2']=df_p2.duplicated(subset=['Subject'])


KeyParamGroup    0
KeyGroup         0
ParamGroup       0
Counts           0
FilePath         0
Subject          0
Multiple_P2      0
dtype: int64

In [259]:
df_m1=df_fmap[df_files['KeyGroup']=='datatype-fmap_fmap-magnitude1_suffix-magnitude1']
df_m1['Multiple_M1']=df_m1.duplicated(subset=['Subject'])
df_m1r=df_m1[df_m1['Multiple_M1']==True]
df_m1r.count()

/scratch/wolfsatterthwaitereward/tmp.WL1HnBKnz7/ipykernel_38573/1223881790.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_m1=df_fmap[df_files['KeyGroup']=='datatype-fmap_fmap-magnitude1_suffix-magnitude1']
/scratch/wolfsatterthwaitereward/tmp.WL1HnBKnz7/ipykernel_38573/1223881790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m1['Multiple_M1']=df_m1.duplicated(subset=['Subject'])


KeyParamGroup    0
KeyGroup         0
ParamGroup       0
Counts           0
FilePath         0
Subject          0
Multiple_M1      0
dtype: int64

In [260]:
df_m2=df_fmap[df_files['KeyGroup']=='datatype-fmap_fmap-magnitude2_suffix-magnitude2']
df_m2['Multiple_M2']=df_m2.duplicated(subset=['Subject'])
df_m2r=df_m2[df_m2['Multiple_M2']==True]
df_m2r.count()

/scratch/wolfsatterthwaitereward/tmp.WL1HnBKnz7/ipykernel_38573/1802827136.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_m2=df_fmap[df_files['KeyGroup']=='datatype-fmap_fmap-magnitude2_suffix-magnitude2']
/scratch/wolfsatterthwaitereward/tmp.WL1HnBKnz7/ipykernel_38573/1802827136.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m2['Multiple_M2']=df_m2.duplicated(subset=['Subject'])


KeyParamGroup    0
KeyGroup         0
ParamGroup       0
Counts           0
FilePath         0
Subject          0
Multiple_M2      0
dtype: int64

In [261]:
#sub with phase1
df_p1.Subject.nunique()

124

In [262]:
#sub with phase2
df_p2.Subject.nunique()

119

In [263]:
#sub with mag1
df_m1.Subject.nunique()

118

In [264]:
#sub with mag2
df_m2.Subject.nunique()

118